## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop('EIN', axis=1, inplace=True)
application_df.drop('NAME', axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
uniquevalues = application_df.nunique()
uniquevalues

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
STATUS,2
INCOME_AMT,9
SPECIAL_CONSIDERATIONS,2
ASK_AMT,8747
IS_SUCCESSFUL,2


In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
ATvaluecounts = application_df['APPLICATION_TYPE'].value_counts()
ATvaluecounts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
CLvaluecounts = application_df['CLASSIFICATION'].value_counts()
CLvaluecounts

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
CLvaluecountsdp = CLvaluecounts[CLvaluecounts > 1]
CLvaluecountsdp

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()

classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")

print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
class_dummies = pd.get_dummies(application_df['CLASSIFICATION'], prefix='CLASS')
class_dummies = class_dummies.astype(int)
application_df = pd.concat([application_df, class_dummies], axis=1)
application_df.drop('CLASSIFICATION', axis=1, inplace=True)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,CLASS_C2100,CLASS_C3000,CLASS_Other
0,T10,Independent,ProductDev,Association,1,0,N,5000,1,1,0,0,0,0,0
1,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,1,0,0,0
2,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0,0,0,0,0,1,0
3,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,0,0,1,0,0,0
4,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,1,0,0,0,0,0


In [10]:
type_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'], prefix='TYPE')
type_dummies = type_dummies.astype(int)
application_df = pd.concat([application_df, type_dummies], axis=1)
application_df.drop('APPLICATION_TYPE', axis=1, inplace=True)
application_df.head()

,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,...,TYPE_Other,TYPE_T10,TYPE_T19,TYPE_T3,TYPE_T4,TYPE_T5,TYPE_T6,TYPE_T7,TYPE_T8,TYPE_T9
0,Independent,ProductDev,Association,1,0,N,5000,1,1,0,...,0,1,0,0,0,0,0,0,0,0
1,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,CompanySponsored,ProductDev,Association,1,0,N,5000,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,1,0,...,0,0,0,1,0,0,0,0,0,0


In [11]:
AFvaluecounts = application_df['AFFILIATION'].value_counts()
AFvaluecounts

affiliations_to_replace = AFvaluecounts[AFvaluecounts < 100].index.tolist()

for aff in affiliations_to_replace:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(aff, "Other")

print(application_df['AFFILIATION'].value_counts())
aff_dummies = pd.get_dummies(application_df['AFFILIATION'], prefix='AFF')
aff_dummies = aff_dummies.astype(int)
application_df = pd.concat([application_df, aff_dummies], axis=1)
application_df.drop('AFFILIATION', axis=1, inplace=True)
application_df.head()


AFFILIATION
Independent         18480
CompanySponsored    15705
Other                 114
Name: count, dtype: int64


,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,...,TYPE_T3,TYPE_T4,TYPE_T5,TYPE_T6,TYPE_T7,TYPE_T8,TYPE_T9,AFF_CompanySponsored,AFF_Independent,AFF_Other
0,ProductDev,Association,1,0,N,5000,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Preservation,Co-operative,1,1-9999,N,108590,1,0,0,1,...,1,0,0,0,0,0,0,0,1,0
2,ProductDev,Association,1,0,N,5000,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,Preservation,Trust,1,10000-24999,N,6692,1,0,0,1,...,1,0,0,0,0,0,0,1,0,0
4,Heathcare,Trust,1,100000-499999,N,142590,1,1,0,0,...,1,0,0,0,0,0,0,0,1,0


In [12]:
UCvaluecounts = application_df['USE_CASE'].value_counts()
UCvaluecounts

,count
USE_CASE,
Preservation,28095
ProductDev,5671
CommunityServ,384
Heathcare,146
Other,3


In [13]:

ucs_to_replace = UCvaluecounts[UCvaluecounts < 1000].index.tolist()

for ucs in ucs_to_replace:
    application_df['USE_CASE'] = application_df['USE_CASE'].replace(ucs, "Other")

print(application_df['USE_CASE'].value_counts())
uc_dummies = pd.get_dummies(application_df['USE_CASE'], prefix='UC')
uc_dummies = uc_dummies.astype(int)
application_df = pd.concat([application_df, uc_dummies], axis=1)
application_df.drop('USE_CASE', axis=1, inplace=True)
application_df.head()


USE_CASE
Preservation    28095
ProductDev       5671
Other             533
Name: count, dtype: int64


,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,CLASS_C2100,...,TYPE_T6,TYPE_T7,TYPE_T8,TYPE_T9,AFF_CompanySponsored,AFF_Independent,AFF_Other,UC_Other,UC_Preservation,UC_ProductDev
0,Association,1,0,N,5000,1,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,Co-operative,1,1-9999,N,108590,1,0,0,1,0,...,0,0,0,0,0,1,0,0,1,0
2,Association,1,0,N,5000,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,Trust,1,10000-24999,N,6692,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
4,Trust,1,100000-499999,N,142590,1,1,0,0,0,...,0,0,0,0,0,1,0,1,0,0


In [14]:
orgvaluecounts = application_df['ORGANIZATION'].value_counts()
orgvaluecounts

,count
ORGANIZATION,
Trust,23515
Association,10255
Co-operative,486
Corporation,43


In [15]:
orgs_to_replace = orgvaluecounts[orgvaluecounts < 1000].index.tolist()

for org in orgs_to_replace:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(org, "Other")

print(application_df['ORGANIZATION'].value_counts())
org_dummies = pd.get_dummies(application_df['ORGANIZATION'], prefix='ORG')
org_dummies = org_dummies.astype(int)
application_df = pd.concat([application_df, org_dummies], axis=1)
application_df.drop('ORGANIZATION', axis=1, inplace=True)
application_df.head()


ORGANIZATION
Trust          23515
Association    10255
Other            529
Name: count, dtype: int64


,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,CLASS_C2100,CLASS_C3000,...,TYPE_T9,AFF_CompanySponsored,AFF_Independent,AFF_Other,UC_Other,UC_Preservation,UC_ProductDev,ORG_Association,ORG_Other,ORG_Trust
0,1,0,N,5000,1,1,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
1,1,1-9999,N,108590,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,0,N,5000,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,1,10000-24999,N,6692,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
4,1,100000-499999,N,142590,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [16]:
spccounts = application_df['SPECIAL_CONSIDERATIONS'].value_counts()
spccounts

,count
SPECIAL_CONSIDERATIONS,
N,34272
Y,27


In [17]:
application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace({'Y': 1, 'N': 0})
application_df.head()


<ipython-input-17-13e959ca2b80>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace({'Y': 1, 'N': 0})


,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,CLASS_C2100,CLASS_C3000,...,TYPE_T9,AFF_CompanySponsored,AFF_Independent,AFF_Other,UC_Other,UC_Preservation,UC_ProductDev,ORG_Association,ORG_Other,ORG_Trust
0,1,0,0,5000,1,1,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
1,1,1-9999,0,108590,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,0,0,5000,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,1,10000-24999,0,6692,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
4,1,100000-499999,0,142590,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [18]:
incomevaluecounts = application_df['INCOME_AMT'].value_counts()
incomevaluecounts

,count
INCOME_AMT,
0,24388
25000-99999,3747
100000-499999,3374
1M-5M,955
1-9999,728
10000-24999,543
10M-50M,240
5M-10M,185
50M+,139


In [19]:
application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace({
    '0': 0,
    '1-9999': 1,
    '10000-24999': 2,
    '25000-99999': 3,
    '100000-499999': 4,
    '1M-5M': 5,
    '5M-10M': 6,
    '10M-50M':7,
    '50M+': 8
   })
application_df.head()

<ipython-input-19-47b7e47c7aad>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace({


,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CLASS_C1000,CLASS_C1200,CLASS_C2000,CLASS_C2100,CLASS_C3000,...,TYPE_T9,AFF_CompanySponsored,AFF_Independent,AFF_Other,UC_Other,UC_Preservation,UC_ProductDev,ORG_Association,ORG_Other,ORG_Trust
0,1,0,0,5000,1,1,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
1,1,1,0,108590,1,0,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,1,0,0,5000,0,0,0,0,0,1,...,0,1,0,0,0,0,1,1,0,0
3,1,2,0,6692,1,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1
4,1,4,0,142590,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
X_train_scaled.shape


(25724, 29)

In [23]:
y_train.shape

(25724,)

## Compile, Train and Evaluate the Model

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=29,))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=29, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 29)                  │             957 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              30 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,947 (7.61 KB)

 Trainable params: 1,947 (7.61 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)


Epoch 1/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.6820 - loss: 0.6159
Epoch 2/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7330 - loss: 0.5558
Epoch 3/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7330 - loss: 0.5515
Epoch 4/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7315 - loss: 0.5502
Epoch 5/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7334 - loss: 0.5524
Epoch 6/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7326 - loss: 0.5528
Epoch 7/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7339 - loss: 0.5519
Epoch 8/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7319 - loss: 0.5478
Epoch 9/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7349 - loss: 0.5482
Epoch 10/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7322 - loss: 0.5513
Epoch 11/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7375 - loss: 0.5438
Epoch 12/50
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 2ms/step - accuracy: 0.7244 - loss: 0.5539
Loss: 0.5538763403892517, Accuracy: 0.7244315147399902


In [28]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity.h5')

In [29]:
# Optimize the model - using BatchNormalization
from keras.layers import Dense, Dropout, BatchNormalization

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=32, activation="relu", input_dim=29,))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=29, activation="relu"))

# Third hidden layer
nn.add(BatchNormalization())

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'BatchNormalization' is not defined